# DML and DMR Analysis

In this notebook, I will examine the location of differentially methylated loci (DML) and regions (DMR) in the *C. virginica* genome. The DML and DMR were identified using methylKit in [this R script](https://github.com/fish546-2018/yaamini-virginica/tree/master/analyses/2018-10-25-MethylKit).

Methods:

0. Set Working Directory
1. Locate Files and Set Variable Paths
2. Identify Overlaps with Genomic Feature Tracks
3. Calculate Overlap Proprtions
4. Gene Flanking
5. Enrichment Analysis

## 0. Set working directory

In [1]:
pwd

'/Users/yaamini/Documents/yaamini-virginica/notebooks'

In [2]:
cd ../analyses/

/Users/yaamini/Documents/yaamini-virginica/analyses


In [3]:
pwd

'/Users/yaamini/Documents/yaamini-virginica/analyses'

In [1]:
!mkdir 2018-11-01-DML-and-DMR-Analysis

In [4]:
ls -F

2018-10-25-MethylKit/            README.md
2018-11-01-DML-and-DMR-Analysis/


In [5]:
cd 2018-11-01-DML-and-DMR-Analysis/

/Users/yaamini/Documents/yaamini-virginica/analyses/2018-11-01-DML-and-DMR-Analysis


## 1. Locate Relevant Files and Set Variable Path Names

### 1a. Set Variable Path Names

Setting the variable path names allows me to reuse this script with different input files or different paths to programs without manually changing the file names each time.

In [6]:
bedtoolsDirectory = "/Users/Shared/bioinformatics/bedtools2/bin/"

In [7]:
DMLlist = "../../analyses/2018-10-25-MethylKit/2018-11-07-DML-Locations.bed"

In [8]:
DMRlist = "../../analyses/2018-10-25-MethylKit/2018-11-07-DMR-Locations.bed"

I will be using the following Genome Feature Tracks that I previously obtained from the [Roberts Lab Genomic Resources wiki page](https://github.com/RobertsLab/resources/wiki/Genomic-Resources) using `curl`:

1. Exon: Coding regions
2. Intron: Regions that are removed
3. mRNA: Code for proteins! The mRNA track includes both introns and exons.
4. Transposable elements (all): Transposable elements located using information from multiple species
5. Tranpsosable elements (_C. virginica_): Transposable elements located using information from _C. virginica_ only
4. CG motifs: Regions with CGs where methylation can occur

In [9]:
exonList = "C_virginica-3.0_Gnomon_exon.bed"

In [10]:
intronList = "C_virginica-3.0_intron.bed"

In [11]:
mRNAList = "C_virginica-3.0_Gnomon_mRNA.gff3"

In [12]:
transposableElementsAll = "C_virginica-3,0_TE-all.gff"

In [13]:
transposableElementsCg = "C_virginica-3.0_TE-Cg.gff"

In [14]:
CGMotifList = "C_virginica-3.0_CG-motif.bed"

### 1b. Confirm Variable Path Works and Characterize Files

The BEDfiles with DML and DMR can be viewed below. Columns are are the chromosome, start position, end position, strand, and fold difference with direction. The files only have DML and DMR that were at least 50% different between the two treatments (control and elevated pCO2).

In [15]:
#Previewing the files
!head {DMLlist}

NC_035780.1	346071	346073	-	50
NC_035780.1	990995	990997	-	-51
NC_035780.1	1882691	1882693	-	52
NC_035780.1	1885022	1885024	-	61
NC_035780.1	1933499	1933501	-	53
NC_035780.1	1945182	1945184	+	55
NC_035780.1	1958998	1959000	-	53
NC_035780.1	1983256	1983258	-	-69
NC_035780.1	2538924	2538926	-	-50
NC_035780.1	2541652	2541654	-	-55


In [16]:
#Counting the number of lines to count DML
!wc -l {DMLlist}

    1398 ../../analyses/2018-10-25-MethylKit/2018-11-07-DML-Locations.bed


In [17]:
!head {DMRlist}

NC_035780.1	571100	571200	DMR	58
NC_035780.1	573700	573800	DMR	52
NC_035780.1	1885000	1885100	DMR	51
NC_035780.1	1933500	1933600	DMR	53
NC_035780.1	4285700	4285800	DMR	-51
NC_035780.1	24159600	24159700	DMR	51
NC_035780.1	26629500	26629600	DMR	65
NC_035780.1	28563400	28563500	DMR	59
NC_035780.1	29883000	29883100	DMR	-55
NC_035780.1	31302900	31303000	DMR	-61


In [18]:
#Counting the number of DMR
!wc -l {DMRlist}

     162 ../../analyses/2018-10-25-MethylKit/2018-11-07-DMR-Locations.bed


In [19]:
!head {exonList}

NC_035780.1	13578	13603
NC_035780.1	14237	14290
NC_035780.1	14557	14594
NC_035780.1	28961	29073
NC_035780.1	30524	31557
NC_035780.1	31736	31887
NC_035780.1	31977	32565
NC_035780.1	32959	33324
NC_035780.1	66869	66897
NC_035780.1	64123	64334


In [20]:
!wc -l {exonList}

  731279 C_virginica-3.0_Gnomon_exon.bed


In [21]:
!head {intronList}

NC_035780.1	28961	28961
NC_035780.1	29074	30524
NC_035780.1	31558	31736
NC_035780.1	31888	31977
NC_035780.1	32566	32959
NC_035780.1	43110	43112
NC_035780.1	44359	45913
NC_035780.1	46507	64123
NC_035780.1	64335	66869
NC_035780.1	85606	85606


In [22]:
!wc -l {intronList}

  319262 C_virginica-3.0_intron.bed


In [23]:
!head {mRNAList}

NC_035780.1	Gnomon	mRNA	28961	33324	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	Gnomon	mRNA	43111	66897	.	-	.	ID=rna2;Parent=gene2;Dbxref=GeneID:111110729,Genbank:XM_022447324.1;Name=XM_022447324.1;gbkey=mRNA;gene=LOC111110729;model_evidence=Supporting evidence includes similarity to: 1 Protein%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments;product=FMRFamide receptor-like%2C transcript variant X1;transcript_id=XM_022447324.1
NC_035780.1	Gnomon	mRNA	43111	46506	.	-	.	ID=rna3;Parent=gene2;Dbxref=GeneID:111110729,Genbank:XM_022447333.1;Name=XM_022447333.1;gbkey=mRNA;gene=LOC111110729;model_evidence=Supporting

In [24]:
!wc -l {mRNAList}

   60201 C_virginica-3.0_Gnomon_mRNA.gff3


In [27]:
!head {transposableElementsAll}

##gff-version 2
##date 2018-08-23
##sequence-region Cvirginica_v300.fa
NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	RepeatMasker	similarity	1728	1947	26.1	-	.	Target "Motif:REP-6_LMi" 14320 14534
NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_007175.2	RepeatMasker	similarity	2129	2367	20.5	-	.	Target "Motif:REP-6_LMi" 13886 14118
NC_007175.2	RepeatMasker	similarity	2836	2980	31.5	+	.	Target "Motif:REP-6_LMi" 6216 6359
NC_007175.2	RepeatMasker	similarity	3196	3277	30.5	+	.	Target "Motif:REP-6_LMi" 6572 6653
NC_007175.2	RepeatMasker	similarity	5168	5532	32.9	+	.	Target "Motif:REP-6_LMi" 4620 4983


In [29]:
!wc -l {transposableElementsAll}

  692371 C_virginica-3,0_TE-all.gff


In [30]:
!head {transposableElementsCg}

##gff-version 2
##date 2018-08-27
##sequence-region Cvirginica_v300.fa
NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_007175.2	RepeatMasker	similarity	6529	6628	19.0	+	.	Target "Motif:(TA)n" 2 102
NC_035780.1	RepeatMasker	similarity	1473	1535	 0.0	+	.	Target "Motif:(TAACCC)n" 1 63
NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	RepeatMasker	similarity	7423	7489	25.4	-	.	Target "Motif:Gypsy-62_CGi-I" 2097 2163
NC_035780.1	RepeatMasker	similarity	7623	8079	34.1	-	.	Target "Motif:Gypsy-62_CGi-I" 1516 1975
NC_035780.1	RepeatMasker	similarity	8261	8295	14.1	+	.	Target "Motif:(CTCCT)n" 1 33


In [31]:
!wc -l {transposableElementsCg}

  626665 C_virginica-3.0_TE-Cg.gff


In [25]:
!head {CGMotifList}

NC_035780.1	28	30	CG_motif
NC_035780.1	54	56	CG_motif
NC_035780.1	75	77	CG_motif
NC_035780.1	93	95	CG_motif
NC_035780.1	103	105	CG_motif
NC_035780.1	116	118	CG_motif
NC_035780.1	134	136	CG_motif
NC_035780.1	159	161	CG_motif
NC_035780.1	209	211	CG_motif
NC_035780.1	224	226	CG_motif


In [26]:
!wc -l {CGMotifList}

 14458703 C_virginica-3.0_CG-motif.bed


## 2. Identify DML and DMR Overlaps with Genomic Feature Tracks

To identify the location of DML and DMR in the *C. virginica* genome, I will use `intersect` from `bedtools`. [The BEDtools suite](http://bedtools.readthedocs.io/en/latest/content/bedtools-suite.html) allows me to easily find overlapping regions of different bed files.

In [36]:
! {bedtoolsDirectory}intersectBed -h


Tool:    bedtools intersect (aka intersectBed)
Version: v2.26.0
Summary: Report overlaps between two feature files.

Usage:   bedtools intersect [OPTIONS] -a <bed/gff/vcf/bam> -b <bed/gff/vcf/bam>

	Note: -b may be followed with multiple databases and/or 
	wildcard (*) character(s). 
Options: 
	-wa	Write the original entry in A for each overlap.

	-wb	Write the original entry in B for each overlap.
		- Useful for knowing _what_ A overlaps. Restricted by -f and -r.

	-loj	Perform a "left outer join". That is, for each feature in A
		report each overlap with B.  If no overlaps are found, 
		report a NULL feature for B.

	-wo	Write the original A and B entries plus the number of base
		pairs of overlap between the two features.
		- Overlaps restricted by -f and -r.
		  Only A features with overlap are reported.

	-wao	Write the original A and B entries plus the number of base
		pairs of overlap between the two features.
		- Overlapping features restricted by -f and -r.
		  However, A fea

### 2a. Exons

#### DML

In [32]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {exonList} \
| wc -l
!echo "DML overlaps with exons"

     786
DML overlaps with exons


In [33]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {exonList} \
> 2018-11-07-DML-Exon.txt

In [34]:
!head 2018-11-07-DML-Exon.txt

NC_035780.1	346071	346073	-	50	NC_035780.1	345983	346125
NC_035780.1	990995	990997	-	-51	NC_035780.1	990854	991062
NC_035780.1	1958998	1959000	-	53	NC_035780.1	1958375	1959139
NC_035780.1	1958998	1959000	-	53	NC_035780.1	1958375	1959139
NC_035780.1	1958998	1959000	-	53	NC_035780.1	1958375	1959139
NC_035780.1	1958998	1959000	-	53	NC_035780.1	1958375	1959139
NC_035780.1	1958998	1959000	-	53	NC_035780.1	1958375	1959139
NC_035780.1	1958998	1959000	-	53	NC_035780.1	1958375	1959139
NC_035780.1	1983256	1983258	-	-69	NC_035780.1	1983248	1983390
NC_035780.1	2538924	2538926	-	-50	NC_035780.1	2538624	2538955


#### DMR

In [35]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMRlist} \
-b {exonList} \
| wc -l
!echo "DMR overlaps with exons"

      64
DMR overlaps with exons


In [36]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMRlist} \
-b {exonList} \
> 2018-11-07-DMR-Exon.txt

In [37]:
!head 2018-11-07-DMR-Exon.txt

NC_035780.1	571100	571194	DMR	58	NC_035780.1	570942	571194
NC_035780.1	573700	573800	DMR	52	NC_035780.1	573630	573906
NC_035780.1	573700	573800	DMR	52	NC_035780.1	573630	573906
NC_035780.1	1933574	1933600	DMR	53	NC_035780.1	1933574	1933615
NC_035780.1	47335000	47335100	DMR	-66	NC_035780.1	47334080	47336192
NC_035780.1	48394800	48394900	DMR	-63	NC_035780.1	48394159	48395287
NC_035780.1	61138200	61138300	DMR	-79	NC_035780.1	61138000	61140417
NC_035781.1	6831300	6831302	DMR	-59	NC_035781.1	6831093	6831302
NC_035781.1	6831300	6831302	DMR	-59	NC_035781.1	6831093	6831302
NC_035781.1	6831300	6831302	DMR	-59	NC_035781.1	6831093	6831302


### 2b. Introns

#### DML

In [38]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {intronList} \
| wc -l
!echo "DML overlaps with introns"

     498
DML overlaps with introns


In [39]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {intronList} \
> 2018-11-07-DML-Intron.txt

In [40]:
!head 2018-11-07-DML-Intron.txt

NC_035780.1	1882691	1882693	-	52	NC_035780.1	1882356	1882972
NC_035780.1	1885022	1885024	-	61	NC_035780.1	1884755	1886043
NC_035780.1	1933499	1933501	-	53	NC_035780.1	1932877	1933574
NC_035780.1	1945182	1945184	+	55	NC_035780.1	1945169	1946107
NC_035780.1	2541652	2541654	-	-55	NC_035780.1	2538956	2541769
NC_035780.1	2541726	2541728	+	-50	NC_035780.1	2538956	2541769
NC_035780.1	2541726	2541728	-	-58	NC_035780.1	2538956	2541769
NC_035780.1	2584492	2584494	+	56	NC_035780.1	2584154	2584505
NC_035780.1	2729868	2729870	-	-50	NC_035780.1	2716215	2733757
NC_035780.1	4288213	4288215	+	-56	NC_035780.1	4288129	4288231


#### DMR

In [41]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMRlist} \
-b {intronList} \
| wc -l
!echo "DMR overlaps with introns"

     112
DMR overlaps with introns


In [42]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMRlist} \
-b {intronList} \
> 2018-11-07-DMR-Intron.txt

In [43]:
!head 2018-11-07-DMR-Intron.txt

NC_035780.1	571195	571200	DMR	58	NC_035780.1	571195	572677
NC_035780.1	1885000	1885100	DMR	51	NC_035780.1	1884755	1886043
NC_035780.1	1933500	1933574	DMR	53	NC_035780.1	1932877	1933574
NC_035780.1	4285700	4285800	DMR	-51	NC_035780.1	4285382	4285831
NC_035780.1	26629500	26629600	DMR	65	NC_035780.1	26621919	26632637
NC_035780.1	28563400	28563500	DMR	59	NC_035780.1	28563400	28564616
NC_035780.1	29883000	29883100	DMR	-55	NC_035780.1	29882984	29883643
NC_035780.1	31302900	31303000	DMR	-61	NC_035780.1	31302842	31303152
NC_035780.1	31303300	31303400	DMR	-53	NC_035780.1	31303293	31303603
NC_035780.1	33209900	33210000	DMR	-54	NC_035780.1	33209785	33210978


### 2c. mRNA

#### DML

In [44]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {mRNAList} \
| wc -l
!echo "DML overlaps with mRNA"

    1263
DML overlaps with mRNA


In [45]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {mRNAList} \
> 2018-11-07-DML-mRNA.txt

In [46]:
!head 2018-11-07-DML-mRNA.txt

NC_035780.1	346071	346073	-	50	NC_035780.1	Gnomon	mRNA	341638	349379	.	-	.	ID=rna30;Parent=gene22;Dbxref=GeneID:111113503,Genbank:XM_022451800.1;Name=XM_022451800.1;gbkey=mRNA;gene=LOC111113503;model_evidence=Supporting evidence includes similarity to: 4 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 2 samples with support for all annotated introns;product=F-box only protein 47-like;transcript_id=XM_022451800.1
NC_035780.1	990995	990997	-	-51	NC_035780.1	Gnomon	mRNA	984471	995318	.	-	.	ID=rna117;Parent=gene66;Dbxref=GeneID:111137104,Genbank:XM_022488366.1;Name=XM_022488366.1;gbkey=mRNA;gene=LOC111137104;model_evidence=Supporting evidence includes similarity to: 1 EST%2C 3 Proteins%2C and 99%25 coverage of the annotated genomic feature by RNAseq alignments;product=SWI/SNF complex subunit SMARCC2-like;transcript_id=XM_022488366.1
NC_035780.1	1882691	1882693	-	52	NC_035780.1	Gnomon	mRNA	1882143	1890106	.	-	.	ID=rna155;Parent=gene95;Dbxre

I know how many overlaps there are, but I also want to know how many unique genes have DMLs in them. For this, I will use the following code:

`cut -f14 2018-11-07-DML-mRNA.txt | sort | uniq -c`

`cut` is the command that isolates the column information. The column is piped into `sort`, then that output is counted for unique lines by `uniq`. I will save the output from this command as a new file.

In [47]:
! cut -f14 2018-11-07-DML-mRNA.txt | sort | uniq -c > 2018-11-07-Unique-Genes-in-DML-mRNA-Overlap.txt

In [48]:
!head 2018-11-07-Unique-Genes-in-DML-mRNA-Overlap.txt

   1 ID=rna10015;Parent=gene5875;Dbxref=GeneID:111118923,Genbank:XM_022458638.1;Name=XM_022458638.1;gbkey=mRNA;gene=LOC111118923;model_evidence=Supporting evidence includes similarity to: 4 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 8 samples with support for all annotated introns;product=copper transporter 2-like;transcript_id=XM_022458638.1
   1 ID=rna10016;Parent=gene5876;Dbxref=GeneID:111118921,Genbank:XM_022458637.1;Name=XM_022458637.1;gbkey=mRNA;gene=LOC111118921;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 24 samples with support for all annotated introns;product=uncharacterized LOC111118921;transcript_id=XM_022458637.1
   1 ID=rna10055;Parent=gene5904;Dbxref=GeneID:111121117,Genbank:XM_022462246.1;Name=XM_022462246.1;gbkey=mRNA;gene=LOC111121117;model_evidence=Supporting evidence includes similarity to: 4 Proteins%2C and 100%25

In [49]:
!wc -l 2018-11-07-Unique-Genes-in-DML-mRNA-Overlap.txt

    2683 2018-11-07-Unique-Genes-in-DML-mRNA-Overlap.txt


The DMLs overlap with 2683 unique genes.

#### DMR

In [50]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMRlist} \
-b {mRNAList} \
| wc -l
!echo "DMR overlaps with mRNA"

     139
DMR overlaps with mRNA


In [51]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMRlist} \
-b {mRNAList} \
> 2018-11-07-DMR-mRNA.txt

In [52]:
!head 2018-11-07-DMR-mRNA.txt

NC_035780.1	571100	571200	DMR	58	NC_035780.1	Gnomon	mRNA	544088	573497	.	+	.	ID=rna48;Parent=gene35;Dbxref=GeneID:111114201,Genbank:XM_022452489.1;Name=XM_022452489.1;Note=The sequence of the model RefSeq transcript was modified relative to this genomic sequence to represent the inferred CDS: inserted 2 bases in 2 codons;exception=unclassified transcription discrepancy;gbkey=mRNA;gene=LOC111114201;model_evidence=Supporting evidence includes similarity to: 4 Proteins%2C and 99%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 9 samples with support for all annotated introns;product=vacuolar protein sorting-associated protein 13B-like;transcript_id=XM_022452489.1
NC_035780.1	573700	573800	DMR	52	NC_035780.1	Gnomon	mRNA	573630	585444	.	-	.	ID=rna49;Parent=gene36;Dbxref=GeneID:111114212,Genbank:XM_022452506.1;Name=XM_022452506.1;gbkey=mRNA;gene=LOC111114212;model_evidence=Supporting evidence includes similarity to: 4 Proteins%2C and 100%25 coverage of the annot

In [53]:
! cut -f14 2018-11-07-DMR-mRNA.txt | sort | uniq -c > 2018-11-07-Unique-Genes-in-DMR-mRNA-Overlap.txt

In [54]:
!head 2018-11-07-Unique-Genes-in-DMR-mRNA-Overlap.txt

   1 ID=rna10182;Parent=gene5989;Dbxref=GeneID:111121400,Genbank:XM_022462674.1;Name=XM_022462674.1;gbkey=mRNA;gene=LOC111121400;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 16 samples with support for all annotated introns;product=DNA excision repair protein ERCC-6-like;transcript_id=XM_022462674.1
   1 ID=rna10216;Parent=gene6005;Dbxref=GeneID:111120829,Genbank:XM_022461817.1;Name=XM_022461817.1;gbkey=mRNA;gene=LOC111120829;model_evidence=Supporting evidence includes similarity to: 1 Protein%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 23 samples with support for all annotated introns;product=serine/arginine-rich splicing factor 7-like%2C transcript variant X1;transcript_id=XM_022461817.1
   1 ID=rna10452;Parent=gene6155;Dbxref=GeneID:111118143,Genbank:XM_022457464.1;Name=XM_022457464.1;Note=The sequence of the model RefSeq tr

In [55]:
!wc -l 2018-11-07-Unique-Genes-in-DMR-mRNA-Overlap.txt

     305 2018-11-07-Unique-Genes-in-DMR-mRNA-Overlap.txt


The DMRs overlap with 305 unique genes.

### 2c. Transposable Elements (All)

#### DML

In [56]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {transposableElementsAll} \
| wc -l
!echo "DML overlaps with transposable elements (all)"

     150
DML overlaps with transposable elements (all)


In [136]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {transposableElementsAll} \
> 2018-11-07-DML-TE-all.txt

In [137]:
!head 2018-11-07-DML-TE-all.txt

NC_035780.1	8833124	8833126	+	62	NC_035780.1	RepeatMasker	similarity	8833042	8833288	18.2	-	.	Target "Motif:CVA" 1 272
NC_035780.1	26599124	26599126	-	-51	NC_035780.1	RepeatMasker	similarity	26598787	26599132	10.1	-	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	30763931	30763933	+	-51	NC_035780.1	RepeatMasker	similarity	30763482	30763996	20.1	+	.	Target "Motif:CVA" 5 465
NC_035780.1	46044690	46044692	+	-56	NC_035780.1	RepeatMasker	similarity	46044523	46044828	 7.0	-	.	Target "Motif:BivaMD-SINE1_CrVi" 37 336
NC_035780.1	46044709	46044711	+	-65	NC_035780.1	RepeatMasker	similarity	46044523	46044828	 7.0	-	.	Target "Motif:BivaMD-SINE1_CrVi" 37 336
NC_035780.1	48579388	48579390	-	-57	NC_035780.1	RepeatMasker	similarity	48579069	48579394	15.3	-	.	Target "Motif:BivaMD-SINE1_CrVi" 1 333
NC_035780.1	57337100	57337102	-	-54	NC_035780.1	RepeatMasker	similarity	57337042	57337128	18.6	-	.	Target "Motif:DNA2-2_CGi" 413 498
NC_035780.1	58135767	58135769	-	74	NC_035780.1	RepeatMasker	similarity

#### DMR

In [59]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMRlist} \
-b {transposableElementsAll} \
| wc -l
!echo "DMR overlaps with transposable elements (all)"

      39
DMR overlaps with transposable elements (all)


In [138]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMRlist} \
-b {transposableElementsAll} \
> 2018-11-07-DMR-TE-all.txt

In [139]:
!head 2018-11-07-DMR-TE-all.txt

NC_035780.1	24159600	24159700	DMR	51
NC_035780.1	29883000	29883100	DMR	-55
NC_035780.1	33945900	33946000	DMR	-51
NC_035780.1	46044700	46044800	DMR	-53
NC_035780.1	47335000	47335100	DMR	-66
NC_035781.1	30962200	30962300	DMR	63
NC_035781.1	51566900	51567000	DMR	-61
NC_035781.1	54151500	54151600	DMR	55
NC_035782.1	2787300	2787400	DMR	-53
NC_035782.1	7518400	7518500	DMR	64


### 2e. Transposable Elements (_C. virginica_ only)

#### DML

In [62]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {transposableElementsCg} \
| wc -l
!echo "DML overlaps with transposable elements (Cv)"

      91
DML overlaps with transposable elements (Cv)


In [63]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {transposableElementsCg} \
> 2018-11-07-DML-TE-Cg.txt

In [64]:
!head 2018-11-07-DML-TE-Cg.txt

NC_035780.1	8833124	8833126	+	62	NC_035780.1	RepeatMasker	similarity	8833045	8833287	22.6	-	.	Target "Motif:Helitron-N2f_CGi" 1 276
NC_035780.1	57337100	57337102	-	-54	NC_035780.1	RepeatMasker	similarity	57337042	57337128	18.6	-	.	Target "Motif:DNA2-2_CGi" 413 498
NC_035781.1	11085270	11085272	+	-54	NC_035781.1	RepeatMasker	similarity	11084936	11085997	29.9	-	.	Target "Motif:Kolobok-11_CGi" 1257 2360
NC_035781.1	12960265	12960267	-	-55	NC_035781.1	RepeatMasker	similarity	12959825	12960341	29.8	+	.	Target "Motif:Gypsy-44_CGi-I" 579 1095
NC_035781.1	21181253	21181255	+	51	NC_035781.1	RepeatMasker	similarity	21180223	21185321	24.4	+	.	Target "Motif:BEL-11_CGi-I" 1086 6234
NC_035781.1	30962237	30962239	+	74	NC_035781.1	RepeatMasker	similarity	30962179	30962531	18.4	+	.	Target "Motif:Helitron-N2d_CGi" 3 346
NC_035781.1	48966784	48966786	+	-57	NC_035781.1	RepeatMasker	similarity	48965730	48967712	25.8	-	.	Target "Motif:Helitron-N2d_CGi" 87 1987
NC_035781.1	54542143	54542145	+	-51	NC_035781.1

#### DMR

In [65]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMRlist} \
-b {transposableElementsCg} \
| wc -l
!echo "DMR overlaps with transposable elements (Cv)"

      23
DMR overlaps with transposable elements (Cv)


In [66]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMRlist} \
-b {transposableElementsCg} \
> 2018-11-07-DMR-TE-Cg.txt

In [67]:
!head 2018-11-07-DMR-TE-Cg.txt

NC_035780.1	29883075	29883100	DMR	-55	NC_035780.1	RepeatMasker	similarity	29883076	29883262	25.4	+	.	Target "Motif:DNA9-4_CGi" 776 964
NC_035780.1	33945935	33945973	DMR	-51	NC_035780.1	RepeatMasker	similarity	33945936	33945973	 5.3	+	.	Target "Motif:DNA9-5_CGi" 1 38
NC_035780.1	33945972	33946000	DMR	-51	NC_035780.1	RepeatMasker	similarity	33945973	33946026	 3.7	+	.	Target "Motif:DNA8-12_CGi" 645 699
NC_035781.1	30962200	30962300	DMR	63	NC_035781.1	RepeatMasker	similarity	30962179	30962531	18.4	+	.	Target "Motif:Helitron-N2d_CGi" 3 346
NC_035781.1	51566900	51566921	DMR	-61	NC_035781.1	RepeatMasker	similarity	51566285	51566921	25.2	-	.	Target "Motif:Sola3-1_CGi" 3273 3922
NC_035781.1	51566900	51567000	DMR	-61	NC_035781.1	RepeatMasker	similarity	51566310	51567155	27.8	-	.	Target "Motif:Helitron-N43_CGi" 1 1221
NC_035781.1	54151500	54151600	DMR	55	NC_035781.1	RepeatMasker	similarity	54150483	54151741	23.3	+	.	Target "Motif:Helitron-N2f_CGi" 1 1018
NC_035782.1	2787300	2787318	DMR	-53	NC_035

## 3. Identify Overlaps between CG Motif and Other Genome Feature Tracks

It's also useful to understand where the CG regions are in relation to exons, introns, mRNA, and transposable elements!

### 3a. Exons

In [68]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {exonList} \
-b {CGMotifList} \
| wc -l
!echo "CG motif overlaps with exons"

  636270
CG motif overlaps with exons


Proportion exon overlap with CG motifs:

In [69]:
636270/14458703

0.04400602184027157

In [71]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {exonList} \
-b {CGMotifList} \
> 2018-11-07-Exon-CGmotif.txt

In [72]:
!head 2018-11-07-Exon-CGmotif.txt

NC_035780.1	13597	13599	NC_035780.1	13597	13599	CG_motif
NC_035780.1	28992	28994	NC_035780.1	28992	28994	CG_motif
NC_035780.1	29001	29003	NC_035780.1	29001	29003	CG_motif
NC_035780.1	29028	29030	NC_035780.1	29028	29030	CG_motif
NC_035780.1	30539	30541	NC_035780.1	30539	30541	CG_motif
NC_035780.1	30574	30576	NC_035780.1	30574	30576	CG_motif
NC_035780.1	30602	30604	NC_035780.1	30602	30604	CG_motif
NC_035780.1	30676	30678	NC_035780.1	30676	30678	CG_motif
NC_035780.1	30695	30697	NC_035780.1	30695	30697	CG_motif
NC_035780.1	30723	30725	NC_035780.1	30723	30725	CG_motif


### 3b. Introns

In [73]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {intronList} \
-b {CGMotifList} \
| wc -l
!echo "CG motif overlaps with introns"

  245500
CG motif overlaps with introns


Proportion intron overlap with CG motifs:

In [74]:
245500/14458703

0.016979392964915317

In [75]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {intronList} \
-b {CGMotifList} \
> 2018-11-07-Intron-CGmotif.txt

In [76]:
!head 2018-11-07-Intron-CGmotif.txt

NC_035780.1	29180	29182	NC_035780.1	29180	29182	CG_motif
NC_035780.1	29203	29205	NC_035780.1	29203	29205	CG_motif
NC_035780.1	29221	29223	NC_035780.1	29221	29223	CG_motif
NC_035780.1	29295	29297	NC_035780.1	29295	29297	CG_motif
NC_035780.1	29323	29325	NC_035780.1	29323	29325	CG_motif
NC_035780.1	29326	29328	NC_035780.1	29326	29328	CG_motif
NC_035780.1	29412	29414	NC_035780.1	29412	29414	CG_motif
NC_035780.1	29452	29454	NC_035780.1	29452	29454	CG_motif
NC_035780.1	29672	29674	NC_035780.1	29672	29674	CG_motif
NC_035780.1	29758	29760	NC_035780.1	29758	29760	CG_motif


### 3c. mRNA

In [77]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {mRNAList} \
-b {CGMotifList} \
| wc -l
!echo "CG motif overlaps with mRNA"

   60195
CG motif overlaps with mRNA


Proportion mRNA overlap with CG motifs:

In [78]:
60195/14458703

0.004163236495002352

In [79]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {mRNAList} \
-b {CGMotifList} \
> 2018-11-07-mRNA-CGmotif.txt

In [80]:
!head 2018-11-07-mRNA-CGmotif.txt

NC_035780.1	Gnomon	mRNA	28993	28994	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1	NC_035780.1	28992	28994	CG_motif
NC_035780.1	Gnomon	mRNA	29002	29003	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1	NC_035780.1	29001	29003	CG_motif
NC_035780.1	Gnomon	mRNA	29029	29030	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:1111269

### 3d. Transposable Elements (All)

In [92]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {transposableElementsAll} \
-b {CGMotifList} \
| wc -l
!echo "CG motif overlaps with transposable elements (all)"

  438365
CG motif overlaps with transposable elements (all)


Proportion TE (all) overlap with CG motifs:

In [93]:
438365/14458703

0.03031841791065215

In [94]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {transposableElementsAll} \
-b {CGMotifList} \
> 2018-11-07-TE-all-CGmotif.txt

In [95]:
!head 2018-11-07-TE-all-CGmotif.txt

NC_007175.2	RepeatMasker	similarity	264	265	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055	NC_007175.2	263	265	CG_motif
NC_007175.2	RepeatMasker	similarity	266	267	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055	NC_007175.2	265	267	CG_motif
NC_007175.2	RepeatMasker	similarity	332	333	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055	NC_007175.2	331	333	CG_motif
NC_007175.2	RepeatMasker	similarity	367	368	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055	NC_007175.2	366	368	CG_motif
NC_007175.2	RepeatMasker	similarity	473	474	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055	NC_007175.2	472	474	CG_motif
NC_007175.2	RepeatMasker	similarity	592	593	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055	NC_007175.2	591	593	CG_motif
NC_007175.2	RepeatMasker	similarity	665	666	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055	NC_007175.2	664	666	CG_motif
NC_007175.2	RepeatMasker	similarity	684	685	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055	NC_007175.2	683	685	CG_motif
NC_007175.2	RepeatMasker	similarity	709	710	31.1	+	.	Target "Mot

### 3e. Transposable Elements (_C. virginca_ only)

In [96]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {transposableElementsCg} \
-b {CGMotifList} \
| wc -l
!echo "CG motif overlaps with transposable elements (Cv)"

  372479
CG motif overlaps with transposable elements (Cv)


Proportion TE (Cv) overlap with CG motifs:

In [97]:
372479/14458703

0.025761577646349055

In [98]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {transposableElementsCg} \
-b {CGMotifList} \
> 2018-11-07-TE-Cv-CGmotif.txt

In [99]:
!head 2018-11-07-TE-Cv-CGmotif.txt

NC_007175.2	RepeatMasker	similarity	1874	1875	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520	NC_007175.2	1873	1875	CG_motif
NC_007175.2	RepeatMasker	similarity	1919	1920	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520	NC_007175.2	1918	1920	CG_motif
NC_007175.2	RepeatMasker	similarity	2004	2005	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520	NC_007175.2	2003	2005	CG_motif
NC_035780.1	RepeatMasker	similarity	5080	5080	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631	NC_035780.1	5078	5080	CG_motif
NC_035780.1	RepeatMasker	similarity	5160	5161	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631	NC_035780.1	5159	5161	CG_motif
NC_035780.1	RepeatMasker	similarity	5163	5164	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631	NC_035780.1	5162	5164	CG_motif
NC_035780.1	RepeatMasker	similarity	5175	5176	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631	NC_035780.1	5174	5176	CG_motif
NC_035780.1	RepeatMasker	similarity	5192	5193	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631	NC_035780.1	5191	5193	CG_moti

## 4. Identify Overlaps between Transposable Elements and Other Genome Feature Tracks

To fully understand my results, I also need to know where TEs are located with respect to exons, introns, and mRNA coding regions.

### 4a. Transposable Elements (All)

#### Exons

In [100]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {exonList} \
-b {transposableElementsAll} \
| wc -l
!echo "Exon overlaps with transposable elements (all)"

   50331
Exon overlaps with transposable elements (all)


Proportion exon overlap with TE (all):

In [101]:
50331/692371

0.07269368589961163

In [102]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {exonList} \
-b {transposableElementsAll} \
> 2018-11-07-Exon-TE-all.txt

In [103]:
!head 2018-11-07-Exon-TE-all.txt

NC_035780.1	109967	109996	NC_035780.1	RepeatMasker	similarity	109968	109996	 0.0	+	.	Target "Motif:(CCT)n" 1 29
NC_035780.1	164885	164914	NC_035780.1	RepeatMasker	similarity	164886	164914	 7.3	+	.	Target "Motif:(GAG)n" 1 29
NC_035780.1	166074	166280	NC_035780.1	RepeatMasker	similarity	166075	166280	32.8	+	.	Target "Motif:Harbinger1_DR" 1472 1676
NC_035780.1	166500	166566	NC_035780.1	RepeatMasker	similarity	166501	166566	30.3	+	.	Target "Motif:Harbinger-6_DR" 1152 1217
NC_035780.1	166597	166642	NC_035780.1	RepeatMasker	similarity	166598	166642	17.8	+	.	Target "Motif:hATw-1_HM" 2778 2822
NC_035780.1	220121	220199	NC_035780.1	RepeatMasker	similarity	220122	220199	24.7	-	.	Target "Motif:Gypsy-75_CQ-I" 1012 1091
NC_035780.1	228341	228392	NC_035780.1	RepeatMasker	similarity	228342	228392	20.0	+	.	Target "Motif:RTE-3_Hmel" 1405 1455
NC_035780.1	227767	227819	NC_035780.1	RepeatMasker	similarity	227768	227819	25.0	+	.	Target "Motif:A-rich" 1 54
NC_035780.1	228341	228392	NC_035780.1	RepeatMasker

#### Introns

In [105]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {intronList} \
-b {transposableElementsAll} \
| wc -l
!echo "Intron overlaps with transposable elements (all)"

  105643
Intron overlaps with transposable elements (all)


Proportion intron overlap with TE (all):

In [106]:
105643/692371

0.1525814917147021

In [107]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {intronList} \
-b {transposableElementsAll} \
> 2018-11-07-Intron-TE-all.txt

In [108]:
!head 2018-11-07-Intron-TE-all.txt

NC_035780.1	32719	32819	NC_035780.1	RepeatMasker	similarity	32720	32819	18.2	+	.	Target "Motif:Crypton-9N1_CGi" 239 337
NC_035780.1	48462	48520	NC_035780.1	RepeatMasker	similarity	48463	48520	 8.8	+	.	Target "Motif:BivaMD-SINE1_CrVi" 280 337
NC_035780.1	48665	49000	NC_035780.1	RepeatMasker	similarity	48666	49000	10.9	-	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	50250	50279	NC_035780.1	RepeatMasker	similarity	50251	50279	 0.0	+	.	Target "Motif:(GGTTAG)n" 1 29
NC_035780.1	50605	50760	NC_035780.1	RepeatMasker	similarity	50606	50760	21.3	+	.	Target "Motif:Harbinger-2N1_CGi" 1 166
NC_035780.1	50976	51034	NC_035780.1	RepeatMasker	similarity	50977	51034	 0.0	+	.	Target "Motif:(TA)n" 1 58
NC_035780.1	51455	51498	NC_035780.1	RepeatMasker	similarity	51456	51498	 0.0	+	.	Target "Motif:(AG)n" 1 43
NC_035780.1	51720	51922	NC_035780.1	RepeatMasker	similarity	51721	51922	21.8	+	.	Target "Motif:Harbinger-2N1_CGi" 2568 2776
NC_035780.1	53155	53294	NC_035780.1	RepeatMasker	similarity	53156	532

#### mRNA

In [109]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {mRNAList} \
-b {transposableElementsAll} \
| wc -l
!echo "mRNA overlaps with transposable elements (all)"

   55069
mRNA overlaps with transposable elements (all)


Proportion mRNA overlap with TE (all):

In [110]:
55069/692371

0.07953683790915564

In [111]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {mRNAList} \
-b {transposableElementsAll} \
> 2018-11-07-mRNA-TE-all.txt

In [112]:
!head 2018-11-07-mRNA-TE-all.txt

NC_035780.1	Gnomon	mRNA	32720	32819	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1	NC_035780.1	RepeatMasker	similarity	32720	32819	18.2	+	.	Target "Motif:Crypton-9N1_CGi" 239 337
NC_035780.1	Gnomon	mRNA	48463	48520	.	-	.	ID=rna2;Parent=gene2;Dbxref=GeneID:111110729,Genbank:XM_022447324.1;Name=XM_022447324.1;gbkey=mRNA;gene=LOC111110729;model_evidence=Supporting evidence includes similarity to: 1 Protein%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments;product=FMRFamide receptor-like%2C transcript variant X1;transcript_id=XM_022447324.1	NC_035780.1	RepeatMasker	similarity	48463	48520	 8.8	+	.	Target "Motif:BivaMD-SINE1_CrVi

### 4b. Transposable Elements (_C. virginica_ only)

#### Exons

In [113]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {exonList} \
-b {transposableElementsCg} \
| wc -l
!echo "Exon overlaps with transposable elements (Cv)"

   41511
Exon overlaps with transposable elements (Cv)


Proportion exon overlap with TE (Cv):

In [115]:
41511/692371

0.059954850795310606

In [116]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {exonList} \
-b {transposableElementsCg} \
> 2018-11-07-Exon-TE-Cv.txt

In [117]:
!head 2018-11-07-Exon-TE-Cv.txt

NC_035780.1	109967	109996	NC_035780.1	RepeatMasker	similarity	109968	109996	 0.0	+	.	Target "Motif:(CCT)n" 1 29
NC_035780.1	164885	164914	NC_035780.1	RepeatMasker	similarity	164886	164914	 7.3	+	.	Target "Motif:(GAG)n" 1 29
NC_035780.1	227767	227819	NC_035780.1	RepeatMasker	similarity	227768	227819	25.0	+	.	Target "Motif:A-rich" 1 54
NC_035780.1	227767	227819	NC_035780.1	RepeatMasker	similarity	227768	227819	25.0	+	.	Target "Motif:A-rich" 1 54
NC_035780.1	227767	227819	NC_035780.1	RepeatMasker	similarity	227768	227819	25.0	+	.	Target "Motif:A-rich" 1 54
NC_035780.1	233475	233478	NC_035780.1	RepeatMasker	similarity	233445	233478	10.1	+	.	Target "Motif:(CCTTT)n" 1 35
NC_035780.1	232863	233028	NC_035780.1	RepeatMasker	similarity	232798	233028	29.7	-	.	Target "Motif:ISL2EU-N8_CGi" 15 237
NC_035780.1	269562	269603	NC_035780.1	RepeatMasker	similarity	269563	269603	17.1	+	.	Target "Motif:(ATG)n" 1 42
NC_035780.1	258539	258574	NC_035780.1	RepeatMasker	similarity	258540	258574	16.3	+	.	Target "

#### Introns

In [118]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {intronList} \
-b {transposableElementsCg} \
| wc -l
!echo "Intron overlaps with transposable elements (Cv)"

   98494
Intron overlaps with transposable elements (Cv)


Proportion intron overlap with TE (Cv):

In [119]:
98494/692371

0.1422561025808418

In [120]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {intronList} \
-b {transposableElementsCg} \
> 2018-11-07-Intron-TE-Cv.txt

In [121]:
!head 2018-11-07-Intron-TE-Cv.txt

NC_035780.1	32719	32819	NC_035780.1	RepeatMasker	similarity	32720	32819	18.2	+	.	Target "Motif:Crypton-9N1_CGi" 239 337
NC_035780.1	46753	46805	NC_035780.1	RepeatMasker	similarity	46754	46805	 6.8	+	.	Target "Motif:DNA-22_CGi" 631 722
NC_035780.1	50250	50279	NC_035780.1	RepeatMasker	similarity	50251	50279	 0.0	+	.	Target "Motif:(GGTTAG)n" 1 29
NC_035780.1	50605	50760	NC_035780.1	RepeatMasker	similarity	50606	50760	21.3	+	.	Target "Motif:Harbinger-2N1_CGi" 1 166
NC_035780.1	50976	51034	NC_035780.1	RepeatMasker	similarity	50977	51034	 0.0	+	.	Target "Motif:(TA)n" 1 58
NC_035780.1	51455	51498	NC_035780.1	RepeatMasker	similarity	51456	51498	 0.0	+	.	Target "Motif:(AG)n" 1 43
NC_035780.1	51720	51922	NC_035780.1	RepeatMasker	similarity	51721	51922	21.8	+	.	Target "Motif:Harbinger-2N1_CGi" 2568 2776
NC_035780.1	86839	86942	NC_035780.1	RepeatMasker	similarity	86840	86942	27.4	-	.	Target "Motif:Helitron-N14_CGi" 83 189
NC_035780.1	87408	87513	NC_035780.1	RepeatMasker	similarity	87409	87513	19.8

#### mRNA

In [122]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {mRNAList} \
-b {transposableElementsCg} \
| wc -l
!echo "mRNA overlaps with transposable elements (Cv)"

   53914
mRNA overlaps with transposable elements (Cv)


Proportion mRNA overlap with TE (Cv):

In [123]:
53914/692371

0.07786865712168765

In [124]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {mRNAList} \
-b {transposableElementsCg} \
> 2018-11-07-mRNA-TE-Cv.txt

In [125]:
!head 2018-11-07-Intron-TE-Cv.txt

NC_035780.1	32719	32819	NC_035780.1	RepeatMasker	similarity	32720	32819	18.2	+	.	Target "Motif:Crypton-9N1_CGi" 239 337
NC_035780.1	46753	46805	NC_035780.1	RepeatMasker	similarity	46754	46805	 6.8	+	.	Target "Motif:DNA-22_CGi" 631 722
NC_035780.1	50250	50279	NC_035780.1	RepeatMasker	similarity	50251	50279	 0.0	+	.	Target "Motif:(GGTTAG)n" 1 29
NC_035780.1	50605	50760	NC_035780.1	RepeatMasker	similarity	50606	50760	21.3	+	.	Target "Motif:Harbinger-2N1_CGi" 1 166
NC_035780.1	50976	51034	NC_035780.1	RepeatMasker	similarity	50977	51034	 0.0	+	.	Target "Motif:(TA)n" 1 58
NC_035780.1	51455	51498	NC_035780.1	RepeatMasker	similarity	51456	51498	 0.0	+	.	Target "Motif:(AG)n" 1 43
NC_035780.1	51720	51922	NC_035780.1	RepeatMasker	similarity	51721	51922	21.8	+	.	Target "Motif:Harbinger-2N1_CGi" 2568 2776
NC_035780.1	86839	86942	NC_035780.1	RepeatMasker	similarity	86840	86942	27.4	-	.	Target "Motif:Helitron-N14_CGi" 83 189
NC_035780.1	87408	87513	NC_035780.1	RepeatMasker	similarity	87409	87513	19.8

## 5. Calculate Overlap Proportions

It's important to understand how many overlaps are present between various feature tracks and CG motifs. CG motifs are where we expect methylation to happen. If there are more overlaps present bewteen a certain feature and the CG motifs, we would expect to see most of our DMLs in that region. I also want to understand overlap proportions with DMLS. 

Here are the questions I will answer:

1. Out the total number of CG motifs, how many overlaped with a feature track?
2. Out of the total number of transposable elements, how many overlaped with a feature track?
2. What proportion of total overlaps does a certain feature track represent?
3. Out the total number of DML, how many overlaped with a feature track?
5. Out of the total number of DMR, how many overlaped with a feature track?

### 5a. CG motif Overlaps with Feature Tracks

I already calculated the numbers associated with the first question in the first section. I'll remind you of those numbers:

- Proportion exon overlap with CG motifs: 4.40% (0.04400602184027157)
- Proportion intron overlap with CG motifs: 1.70% (0.016979392964915317)
- Proportion mRNA overlap with CG motifs: 0.42% (0.004163236495002352)
- Proportion transposable element (all) overlap with CG motifs: 2.58% (0.025761577646349055) 
- Proportion transposable element (_C. virginica_) overlap with CG motifs: 3.03% (0.03031841791065215)

### 5b. Transposable Element (all) Overlaps with Feature Tracks

See 4a for more details.

- Proportion exon overlap with TE (all): 7.27% (0.07269368589961163)
- Proportion intron overlap with TE (all): 15.3% (0.1525814917147021)
- Proportion mRNA overlap with TE (all): 7.95% (0.07953683790915564)

### 5c. Transposable Element (_C. virginica_ only) Overlaps with Feature Tracks

See 4b for more details.

- Proportion exon overlap with TE (Cv): 6.00% (0.059954850795310606)
- Proportion intron overlap with TE (Cv): 14.2% (0.1422561025808418)
- Proportion mRNA overlap with TE (Cv): 7.79% (0.07786865712168765)

### 5d. Proportion Total Overlaps by Feature Track

Since I have two different transposable element tracks, I'll repeat these calculations for each track.

#### Transposable Elements (all)

First, I need to calculate the total number of overlaps we had:

(exon overlap with CG motifs) + (intron overlap with CG motifs) + (mRNA overlap with CG motifs) + (TE overlap with CG motifs)

In [126]:
636270 + 245500 + 60195 + 438365

1380330

Now, I calculate the proportions:

Exons:

In [127]:
636270/1380330

0.4609549890243637

Introns:

In [128]:
245500/1380330

0.17785601993726138

mRNA:

In [129]:
60195/1380330

0.04360913694551303

Transposable Elements (all):

In [130]:
438365/1380330

0.3175798540928619

- Proportion exon overlap out of total overlaps: 46.10% (0.4609549890243637)
- Proportion intron overlap out of total overlaps: 17.79% (0.17785601993726138)
- Proportion mRNA overlap out of total overlaps: 4.36% (0.04360913694551303)
- Proportion transposable elements (all) overlap out of total overlaps: 31.76% (0.3175798540928619)

#### Transposable Elements (_C. virginica_ only)

Total number of overlaps:

In [131]:
636270 + 245500 + 60195 + 372479

1314444

Exons:

In [132]:
636270/1314444

0.4840601805782521

Introns:

In [133]:
245500/1314444

0.18677098453794913

mRNA:

In [134]:
60195/1314444

0.04579502816399938

Tranposable Elements (_C. virginica_ only):

In [135]:
372479/1314444

0.2833738067197994

- Proportion exon overlap out of total overlaps: 48.41% (0.4840601805782521)
- Proportion intron overlap out of total overlaps: 18.68% (0.18677098453794913)
- Proportion mRNA overlap out of total overlaps: 4.58% (0.04579502816399938)
- Proportion transposable elements (Cv) overlap out of total overlaps: 28.34% (0.2833738067197994)

### 5e. DML Overlaps with Feature Tracks

Exons:

In [47]:
786/1398

0.5622317596566524

Introns:

In [46]:
498/1398

0.3562231759656652

mRNA:

In [45]:
1263/1398

0.9034334763948498

Transposable Elements (all):

In [140]:
150/1398

0.1072961373390558

Transposable elements (_C. virginica_ only):

In [141]:
91/1398

0.06509298998569385

- Proportion exon overlap with DMLs: 56.22% (0.5622317596566524)
- Proportion intron overlap with DMLs: 35.62% (0.3562231759656652)
- Proportion mRNA overlap with DMLs: 90.34% (0.9034334763948498)
- Proportion transposable element (all) overlap with DMLs: 10.73% (0.1072961373390558)
- Proportion transposable element (_C. virginica_) overlap with DMLs: 6.51% (0.06509298998569385)

### 5f. DMR Overlaps with Feature Tracks

Exons:

In [19]:
64/162

0.3950617283950617

Introns:

In [20]:
112/162

0.691358024691358

mRNA:

In [21]:
139/162

0.8580246913580247

Transposable Elements (All):

In [142]:
39/162

0.24074074074074073

Transposable Elements (_C. virginica_ only)

In [143]:
23/162

0.1419753086419753

- Proportion exon overlap with DMRs: 39.51% (0.3950617283950617)
- Proportion intron overlap with DMRs: 69.14% (0.691358024691358)
- Proportion mRNA overlap with DMRs: 85.80% (0.8580246913580247)
- Proportion transposable element (all) overlap with DMRs: 24.07% (0.24074074074074073)
- Proportion transposable element (_C. virginica_ only) overlap with DMRs: 14.20% (0.1419753086419753)

# STILL DEVELOPING THE CONTENT BELOW NO GUARANTEES

## 6. Gene Flanking

### 6a. `closest`

After talking to Mac at PCSGA 2018, she suggested using BEDtools [`closest`](https://bedtools.readthedocs.io/en/latest/content/tools/closest.html) instead of `flank`. `closest` will find the nearest genomic feature, but not necessarily a non-overlapping feature. I can modify the code as follows:

1. Path to `closestBed`
2. -io: Ignore features in b that overlap with a
3. -a: Path to mRNA gff
4. -b: Specify either DML or CG motif file
5. ">" filename: Redirect output to a .txt file

In [6]:
! {bedtoolsDirectory}closestBed \
-io \
-a C_virginica-3.0_Gnomon_mRNA.gff3 \
-b ../2018-05-29-MethylKit-Full-Samples/2018-05-30-DML-Locations.bed \
> 2018-09-26-mRNA-Closest-NoOverlap-DMLs.txt

Error: Sorted input specified, but the file C_virginica-3.0_Gnomon_mRNA.gff3 has the following out of order record
NC_035780.1	Gnomon	mRNA	2413594	2416601	.	-	.	ID=rna199;Parent=gene122;Dbxref=GeneID:111129373,Genbank:XM_022475729.1;Name=XM_022475729.1;gbkey=mRNA;gene=LOC111129373;model_evidence=Supporting evidence includes similarity to: 2 Proteins;product=mucin-2-like;transcript_id=XM_022475729.1


In [7]:
! {bedtoolsDirectory}closestBed \
-io \
-a C_virginica-3.0_Gnomon_mRNA.gff3 \
-b C_virginica-3.0_CG-motif.bed \
> 2018-09-26-mRNA-Closest-NoOverlap-CGmotifs.txt

Error: Sorted input specified, but the file C_virginica-3.0_Gnomon_mRNA.gff3 has the following out of order record
NC_035780.1	Gnomon	mRNA	2413594	2416601	.	-	.	ID=rna199;Parent=gene122;Dbxref=GeneID:111129373,Genbank:XM_022475729.1;Name=XM_022475729.1;gbkey=mRNA;gene=LOC111129373;model_evidence=Supporting evidence includes similarity to: 2 Proteins;product=mucin-2-like;transcript_id=XM_022475729.1


The file was created, but the mRNA file itself is unsorted. I need to see if this impacted the output.

## 7. Gene Enrichment Analysis

See this [R Markdown File](https://github.com/RobertsLab/project-virginica-oa/blob/master/analyses/2018-06-11-DML-Analysis/2018-06-14-Gene-Enrichment-Analysis.Rmd) for Gene Enrichment Analysis information.